In [2]:
import os
from datetime import datetime
from collections import deque
import time as t
import pickle
from cassiopeia import riotapi
from cassiopeia.type.api.exception import APIError
from cassiopeia.type.core.common import LoadPolicy
from set_up import set_up, set_up2, set_up3

In [3]:
set_up('KR')

In [4]:
def auto_retry(api_call_method):
    """ A decorator to automatically retry 500s (Service Unavailable)
    and skip 400s (Bad Request) or 404s (Not Found). """
    def call_wrapper(*args, **kwargs):
        try:
            return api_call_method(*args, **kwargs)
        except APIError as error:
            # Try Again Once
            if error.error_code in [500]:
                try:
                    print("Got a 500, trying again...")
                    return api_call_method(*args, **kwargs)
                except APIError as another_error:
                    if another_error.error_code in [500, 400, 404]:
                        pass
                    elif another_error.error_code in [503]:
                        print('Found a 503')
                        t.sleep(120)
                    else:
                        raise another_error

            # Skip
            elif error.error_code in [400, 404]:
                print("Got a 400 or 404")
                pass

            # Fatal
            else:
                raise error
    return call_wrapper

In [5]:
riotapi.get_match = auto_retry(riotapi.get_match)
riotapi.get_summoner_by_id = auto_retry(riotapi.get_summoner_by_id)
riotapi.get_summoner_by_name = auto_retry(riotapi.get_summoner_by_name)

In [29]:
unpulled_summoners = deque(entry.summoner for entry in riotapi.get_challenger())

In [30]:
gather_start = datetime(2017, 7, 12)
match_set = set()
total_req = 0

In [31]:
pulled_summoners = deque()
gather_start = datetime(2017, 7, 12)
match_set = set()
i = 1
while len(unpulled_summoners) > 0:
    try:
        if i in range(1,201,3):
            set_up('KR')
            print('set_up1')
        elif i in range(2,201,3):
            set_up2('KR')
            print('set_up2')
        elif i in range(3,201,3):
            set_up3('KR')
            print('set_up3')
        summoner = unpulled_summoners.popleft()
        summMatchList = summoner.match_list(begin_time=gather_start)
        print('summMatchList' , len(summMatchList))
        for match_reference in summMatchList:
        # If you are connected to a database, the match will automatically be stored in it.
        # Simply pull the match, and it's in your database for whenever you need it again!
        # If you pull a match twice, the second time it will be loaded from the database rather than pulled from Riot
        # and therefore will not count against your rate limit. This is true of all datatypes, include Summoner.
            match = riotapi.get_match(match_reference)
            # t.sleep(15)
            if match is None:  # If the match still fails to load, continue on to the next one
                continue
            elif str(match) in match_set:
                print('Found duplicate' + match)
                continue
            else:
                match_set.add(match)
                print("Stored {0} in my database".format(match))
                #t.sleep(15)
                with open('matches.txt', 'a+') as handle:
                    handle.write(str(match)+ ' ')
        pulled_summoners.append(summoner)
        i+=1
        print(i, 'index')
        #print('Sleeping')
        #t.sleep(60)
        print(len(pulled_summoners), 'pulled summoner')
    except:
        print('503')
        t.sleep(30)
        continue
print('Done')

set_up1


summMatchList 21
Stored Match #2882982944 in my database
Stored Match #2882929189 in my database
Stored Match #2882892989 in my database
Stored Match #2882829445 in my database
Stored Match #2882816431 in my database
Stored Match #2882842648 in my database
Stored Match #2882758496 in my database
Stored Match #2882734819 in my database
Stored Match #2882732493 in my database
Stored Match #2881884027 in my database
Stored Match #2881890354 in my database
Stored Match #2881807909 in my database
Stored Match #2881804167 in my database
Stored Match #2881429397 in my database
Stored Match #2881427901 in my database
Stored Match #2880927873 in my database
Stored Match #2880934633 in my database
Stored Match #2880911057 in my database
Stored Match #2880838008 in my database
Stored Match #2880843175 in my database
Stored Match #2880821436 in my database
2 index
1 pulled summoner
set_up2


summMatchList 3
Stored Match #2882251911 in my database
Stored Match #2881234412 in my database
Stored Match #2881211660 in my database
3 index
2 pulled summoner
set_up3


summMatchList 1
Stored Match #2882912190 in my database
4 index
3 pulled summoner
set_up1


summMatchList 27
Stored Match #2883083986 in my database
Stored Match #2882938203 in my database
Stored Match #2882905690 in my database
Stored Match #2882910507 in my database
Stored Match #2882836658 in my database
Stored Match #2882842648 in my database
Stored Match #2882718201 in my database
Stored Match #2882284837 in my database
Stored Match #2882274096 in my database
Stored Match #2882242932 in my database
Stored Match #2882280212 in my database
Stored Match #2882188271 in my database
Stored Match #2882175104 in my database
Stored Match #2882163015 in my database
Stored Match #2882088003 in my database
Stored Match #2881962364 in my database
Stored Match #2881899218 in my database
Stored Match #2881294128 in my database
Stored Match #2881239941 in my database
Stored Match #2881238652 in my database
Stored Match #2881216940 in my database
Stored Match #2881214802 in my database
Stored Match #2881144398 in my database
Stored Match #2881019474 in my database


Stored Match #2881025752 in my database
Stored Match #2881031376 in my database
Stored Match #2880937574 in my database
5 index
4 pulled summoner
set_up2


summMatchList 8
Stored Match #2881398931 in my database
Stored Match #2881397487 in my database
Stored Match #2881425695 in my database
Stored Match #2881321035 in my database
Stored Match #2881239941 in my database
Stored Match #2881238652 in my database
Stored Match #2881216111 in my database
Stored Match #2881233229 in my database
6 index
5 pulled summoner
set_up3


summMatchList 0
7 index
6 pulled summoner
set_up1


summMatchList 18
Stored Match #2883083986 in my database
Stored Match #2881324335 in my database
Stored Match #2881332964 in my database
Stored Match #2881311741 in my database
Stored Match #2881300181 in my database
Stored Match #2881228224 in my database
Stored Match #2881234183 in my database
Stored Match #2881144398 in my database
Stored Match #2881140474 in my database
Stored Match #2881045004 in my database
503
